##  Mount or Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##  Set Work Directory (Dataset Directory)

In [1]:
%cd '/content/gdrive/MyDrive/myDetection/ShapeRecogition/N5/OnePose_Plus_Plus/2024/CO2/NewApproach'

/content/gdrive/MyDrive/myDetection/ShapeRecogition/N5/OnePose_Plus_Plus/2024/CO2/NewApproach


## Install necessary libraries

In [ ]:
!pip install xlsxwriter

In [11]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2
!pip install matplotlib==3.8.1

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)


In [ ]:
!pip install scikeras

!pip install --upgrade scikit-learn
!pip install xlwt

!pip install openpyxl
!sudo apt install font-manager
!rm ~/.cache/matplotlib -fr

In [ ]:
!pip install --upgrade pandas

## Install necessary font (time New Roman)

In [2]:
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!sudo cp 'Times New Roman.ttf' '/usr/share/fonts/truetype/'

In [ ]:
!sudo fc-cache -fv

In [5]:
!fc-list | grep "Times New Roman"

/usr/share/fonts/truetype/Times New Roman.ttf: Times New Roman:style=Regular,Normal,obyčejné,Standard,Κανονικά,Normaali,Normál,Normale,Standaard,Normalny,Обычный,Normálne,Navadno,thường,Arrunta


In [6]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'  # or 'sans-serif', 'monospace'
# or
plt.rcParams['font.serif'] = ['DejaVu Serif'] #specifies a specific serif font.
plt.rcParams['font.family'] = 'Liberation Serif'  # Another alternative

import matplotlib.font_manager as fm
fm.fontManager.addfont('Times New Roman.ttf')  # Update the path
plt.rcParams['font.family'] = 'Times New Roman'


## Generate the cleaned CSV file

In [ ]:
dataAll, n_exp = read_data('SequentialDataInhibitor_large', new=True)

## Import necessary libraries and Config

In [2]:
import os
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.svm import SVR
from sklearn.utils import shuffle
from sklearn.inspection import permutation_importance
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Add , LayerNormalization
from cuml.svm import SVR as cuSVR
from cuml.ensemble import RandomForestRegressor as cuRF

matplotlib.use('Agg')
matplotlib.rcParams['font.family'] = "Times New Roman"
matplotlib.rcParams['axes.linewidth'] = 1.5
target = {'regression': 'corrosion_mm_yr'}

## Variables

In [3]:
param = dict(test_size=0.25, cv=5, scoring='mse', iterations=2,
             summary= True, grid_search=True , compare=True, importance=True, parity=True,
             production=True, sensitivity=True)

input_dim = 1

## Implemented Normal Functions

In [4]:
def stack_data(df, _set):
    conc = df['concentration_ppm'].unique()
    df2 = pd.DataFrame()
    i = 0
    for c in conc:
        if c == conc[0]:
            df2 = df.loc[df['concentration_ppm'] == c].reset_index(drop=True)
            df2['time_hrs_original'] = df2['time_hrs']
            _min, _max = df2['time_hrs'].min(), df2['time_hrs'].max()
            df2['time_hrs'] = df2['time_hrs'] - _min
            df2['pre_concentration_zero'] = 'Yes'
            df2['pre_concentration_ppm'] = 0
            if _set == 'training':
                df2['initial_corrosion_mm_yr'] = df.loc[0, 'corrosion_mm_yr']
        else:
            df3 = df.loc[df['concentration_ppm'] == c].reset_index(drop=True)
            df3['time_hrs_original'] = df3['time_hrs']
            _min, _max = df3['time_hrs'].min(), df3['time_hrs'].max()
            df3['time_hrs'] = df3['time_hrs'] - _min
            if i == 1:
                df3['pre_concentration_zero'] = 'Yes'
            else:
                df3['pre_concentration_zero'] = 'No'
            df3['pre_concentration_ppm'] = conc[i - 1]
            if _set == 'training':
                df3['initial_corrosion_mm_yr'] = df.loc[0, 'corrosion_mm_yr']
            df2 = pd.concat([df2, df3], ignore_index=True)
        i += 1
    return df2


def read_exp(df, _set):
    df.columns = df.columns.str.replace(', ', '_')
    df.columns = df.columns.str.replace(' ', '_')
    replicas = df['Description'].unique()
    df2 = pd.DataFrame()
    for replica in replicas:
        if replica == replicas[0]:
            df2 = df.loc[df['Description'] == replica].reset_index(drop=True)
            df2 = stack_data(df2, _set)
        else:
            df3 = df.loc[df['Description'] == replica].reset_index(drop=True)
            df3 = stack_data(df3, _set)
            df2 = pd.concat([df2, df3], ignore_index=True)
    return df2


def clean_data(df):
    df = df[df['corrosion_mm_yr'] >= 0.0]
    aux, aux2 = np.log10(df['corrosion_mm_yr']), np.log10(df['initial_corrosion_mm_yr'])
    df = df.drop(['corrosion_mm_yr', 'initial_corrosion_mm_yr'], axis=1)
    df['corrosion_mm_yr'], df['initial_corrosion_mm_yr'] = aux, aux2
    df = df.dropna(axis=0, how='any').reset_index(drop=True)
    df['Lab'] = df['Lab'].str.rstrip()
    df['Type_of_test'] = df['Type_of_test'].str.rstrip()
    df = df.replace({'Type_of_test': {'Sequential Dose': 'sequential_dose',
                                      'Single Dose YP': 'single_dose_YP',
                                      'Single Dose NP': 'single_dose_NP'},
                     'pH': {6: 'Controlled=6'}})
    return df


def read_data(file_name, new):
    if new:
        sheet_names = pd.ExcelFile('{}.xlsx'.format(file_name)).sheet_names
        df = pd.DataFrame()
        n = 0
        for sheet_name in sheet_names:
            if sheet_name == sheet_names[0]:
                df = pd.read_excel('{}.xlsx'.format(file_name), sheet_name=sheet_name)
                df = read_exp(df, 'training')
                df['Experiment'] = n + 1
            else:
                df2 = pd.read_excel('{}.xlsx'.format(file_name), sheet_name=sheet_name)
                df2 = read_exp(df2, 'training')
                df2['Experiment'] = n + 1
                df = pd.concat([df, df2], ignore_index=True)
            n += 1
            print(n)
        df = clean_data(df)
        excel_output(df, _root='', file_name='{}Cleaned'.format(file_name), csv=True)
    else:
        df = pd.read_csv('{}Cleaned.csv'.format(file_name))
        df = df.drop(['Unnamed: 0'], axis=1)
        n = len(df['Experiment'].unique())
    return df, n


def filter_lab(df, lab):
    if lab != 'All':
        df = df[df['Lab'] == lab].reset_index(drop=True)
    return df


def update_data(df, lab):
    df2 = filter_lab(df, lab)
    return df2


def columns_stats(df, _set, _root):
    statistics = pd.DataFrame()
    for column in df.columns:
        if (column != 'time_hrs') and (column != 'time_hrs_original') and \
                (column != 'corrosion_mm_yr') and (column != 'initial_corrosion_mm_yr'):
            if column == df.columns[0]:
                statistics = pd.DataFrame(df[column].value_counts()).reset_index(drop=False)
                statistics.rename(columns={'index': column, column: 'Num_samples'}, inplace=True)
            else:
                temp = pd.DataFrame(df[column].value_counts()).reset_index(drop=False)
                temp.rename(columns={'index': column, column: 'Num_samples'}, inplace=True)
                statistics = pd.concat([statistics, temp], axis=1)
    excel_output(statistics, _root=_root, file_name='columnsStats_{}'.format(_set), csv=False)


def experiments_stats(df, _set, _root):
    statistics = pd.DataFrame(columns=['Experiment', 'num_replica', 'CI concentration (ppm, hrs)', 'Length_hrs',
                                       'Pressure_bar_CO2', 'Temperature_C', 'CI', 'Shear_Pa',
                                       'Brine_Ionic_Strength', 'pH', 'Brine_Type', 'Type_of_test', 'Lab'])
    _experiments = df['Experiment'].unique()
    for _exp in _experiments:
        df2 = df.loc[df['Experiment'] == _exp].reset_index(drop=True)
        df3 = df2.groupby('concentration_ppm')['time_hrs'].max()
        n_replica = len(df2['Description'].unique())
        conc = df2['concentration_ppm'].unique()
        _conc_ppm = ''
        for c in conc:
            tt = df3.loc[c]
            if c == conc[0]:
                _conc_ppm = _conc_ppm + '({:.0f}, {:.0f})'.format(c, tt)
            else:
                _conc_ppm = _conc_ppm + ' - ({:.0f}, {:.0f})'.format(c, tt)
        statistics = statistics._append({'Experiment': _exp,
                                        'num_replica': n_replica,
                                        'CI concentration (ppm, hrs)': _conc_ppm,
                                        'Length_hrs': '~ {:.0f}'.format(df3.sum()),
                                        'Pressure_bar_CO2': df2.loc[0, 'Pressure_bar_CO2'],
                                        'Temperature_C': df2.loc[0, 'Temperature_C'],
                                        'CI': df2.loc[0, 'CI'],
                                        'Shear_Pa': df2.loc[0, 'Shear_Pa'],
                                        'Brine_Ionic_Strength': df2.loc[0, 'Brine_Ionic_Strength'],
                                        'pH': df2.loc[0, 'pH'],
                                        'Brine_Type': df2.loc[0, 'Brine_Type'],
                                        'Type_of_test': df2.loc[0, 'Type_of_test'],
                                        'Lab': df2.loc[0, 'Lab']}, ignore_index=True)

    excel_output(statistics, _root=_root, file_name='experimentsStats_{}'.format(_set), csv=False)


def view_data_exp(df, y_axis_scale, _set, _root):
    _root = '{}/{}{}'.format(_root, _set, y_axis_scale)
    if not os.path.exists(_root):
        os.makedirs(_root)
    # ---------------------------------
    _experiments = df['Experiment'].unique()
    for _exp in _experiments:
        df2 = df.loc[df['Experiment'] == _exp]
        replicas = df2['Description'].unique()
        fig, ax = plt.subplots(1, figsize=(9, 9))
        _X_plot = pd.Series(dtype='float64')
        n = 1
        for rep in replicas:
            df3 = df2.loc[df['Description'] == rep]
            _X = df3['time_hrs_original']
            _y = 10 ** (df3['corrosion_mm_yr'])
            plt.scatter(_X, _y, label='Replica {}'.format(n))
            if n == 1:
                _X_plot = _X
            n += 1
        if y_axis_scale == 'Log':
            plt.yscale('log')
            ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
            if _exp == 14:
                ax.set_ylim(0.001, 100)
                ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
            else:
                ax.set_ylim(0.01, 100)
        # ---------------------------------
        plt.text(0.02, 1.03, 'Experiment {}'.format(_exp),
                 ha='left', va='center', transform=ax.transAxes, fontdict={'color': 'k', 'weight': 'bold', 'size': 21})
        # ---------------------------------
        plt.grid(linewidth=0.5)
        x_axis_max = 10 * (1 + int(np.max(_X_plot) / 10))
        if _exp == 6:
            x_axis_max = 40
        elif _exp == 11 or _exp == 13 or _exp == 17 or _exp == 18 or _exp == 19:
            x_axis_max = 25
        elif _exp == 14:
            x_axis_max = 30
        elif _exp == 16:
            x_axis_max = 15
        x_axis_index = np.linspace(0, x_axis_max, num=6)
        ax.set_xticks(x_axis_index)
        ax.set_xlim(0, x_axis_max)
        ax.set_xticklabels(x_axis_index, fontsize=20)
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.set_xlabel('Time (hr)', fontsize=27)
        plt.yticks(fontsize=20)
        ax.set_ylabel('Corrosion Rate (mm/year)', fontsize=27)
        n_col, leg_fontsize = 1, 20
        if _exp == 10 or _exp == 14:
            n_col, leg_fontsize = 2, 18
        plt.legend(loc='upper right', fontsize=leg_fontsize, ncol=n_col, fancybox=True, shadow=True)
        plt.tight_layout()
        plt.savefig('{}/exp{}.png'.format(_root, _exp))
        plt.close()


def experiments_types(df, y_axis_scale, _experiments, _root):
    _root = '{}/experimentsTypes{}'.format(_root, y_axis_scale)
    if not os.path.exists(_root):
        os.makedirs(_root)
    # ---------------------------------
    for _e in _experiments:
        df2 = df.loc[df['Experiment'] == _e[0]]
        df3 = df2.loc[df['Description'] == _e[1]]
        fig, ax = plt.subplots(1, figsize=(9, 9))
        _X = df3['time_hrs_original'].to_numpy()
        _y = 10 ** (df3['corrosion_mm_yr'].to_numpy())
        marker_size = [50 + i * 0 for i in _y]
        plt.scatter(_X, _y, s=marker_size, c='black')
        if y_axis_scale == 'Log':
            plt.yscale('log')
            ax.set_ylim(0.01, 100)
            plt.yticks(fontsize=20)
        else:
            if _e[0] == 3:
                y_axis_mas = 40
            elif _e[0] == 20:
                y_axis_mas = 10
            else:
                y_axis_mas = 6
            y_axis_index = np.linspace(0, y_axis_mas, num=6)
            ax.set_yticks(y_axis_index)
            ax.set_ylim(0, y_axis_mas)
            ax.set_yticklabels(y_axis_index, fontsize=20)
            ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        # ---------------------------------
        plt.text(0.02, 1.03, '{}'.format(_e[2]),
                 ha='left', va='center', transform=ax.transAxes, fontdict={'color': 'k', 'weight': 'bold', 'size': 21})
        # ---------------------------------
        plt.grid(linewidth=0.5)
        x_axis_index = np.linspace(0, 10 * (1 + int(np.max(_X) / 10)), num=6)
        ax.set_xticks(x_axis_index)
        ax.set_xlim(0, 10 * (1 + int(np.max(_X) / 10)))
        ax.set_xticklabels(x_axis_index, fontsize=20)
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.set_xlabel('Time (hr)', fontsize=27)
        ax.set_ylabel('Corrosion Rate (mm/year)', fontsize=27)
        plt.tight_layout()
        plt.savefig('{}/exp{}.png'.format(_root, _e[0]))
        plt.close()


def summary_data(df):
    _root = 'regression/dataSummary'
    if not os.path.exists(_root):
        os.makedirs(_root)
    # ---------------------------------
    columns_stats(df, 'allReplicas', _root)
    experiments_stats(df, 'allReplicas', _root)
    view_data_exp(df, 'Log', 'allReplicas', _root)
    view_data_exp(df, 'Normal', 'allReplicas', _root)


def excel_output(_object, _root, file_name, csv):
    if csv:
        if _root != '':
            _object.to_csv('{}/{}.csv'.format(_root, file_name))
        else:
            _object.to_csv('{}.csv'.format(file_name))
    else:
        if _root != '':
            _object.to_excel('{}/{}.xlsx'.format(_root, file_name))
        else:
            _object.to_excel('{}.xlsx'.format(file_name))

## Implemented Models and Dataset Functionss  

In [5]:
def select_features(df):
    df = df[['concentration_ppm', 'pre_concentration_ppm', 'time_hrs','time_hrs_original',
             'Pressure_bar_CO2', 'Temperature_C', 'CI', 'Shear_Pa', 'Brine_Ionic_Strength',
             'pH', 'Brine_Type', 'Type_of_test', 'initial_corrosion_mm_yr', 'Description', 'Experiment',
             'corrosion_mm_yr']]
    return df


def encode_data(df):
    cat_index = ['CI', 'pH', 'Brine_Type', 'Type_of_test']
    num_index = ['Pressure_bar_CO2', 'Temperature_C', 'Shear_Pa', 'Brine_Ionic_Strength']
    ohe = ohe = OneHotEncoder(handle_unknown='ignore')
    sc = StandardScaler()
    ct = make_column_transformer((ohe, cat_index), (sc, num_index), remainder='passthrough')
    ct.fit_transform(df)
    df2 = ct.transform(df)
    # ---------------------------------
    names = []
    for cat in cat_index:
        unique = df[cat].value_counts().sort_index()
        for name in unique.index:
            names.append('{}_{}'.format(cat, name))
    for num in num_index:
        names.append(num)
    names.append('concentration_ppm')
    names.append('pre_concentration_ppm')
    names.append('time_hrs')
    names.append('time_hrs_original')
    names.append('initial_corrosion_mm_yr')
    names.append('Description')
    names.append('Experiment')
    names.append('corrosion_mm_yr')
    # ---------------------------------
    df2 = pd.DataFrame(df2)
    df2.columns = names
    return df2


def split_data_random(df):
    df = df.copy(deep=True)
    df = shuffle(df)
    head = int((1 - param['test_size']) * len(df))
    tail = len(df) - head
    df_train = df.head(head).reset_index(drop=True)
    df_test = df.tail(tail).reset_index(drop=True)
    return df_train, df_test


def split_xy(df, _shuffle):
    if _shuffle:
        df = shuffle(df)
    df = df.drop(['Description', 'Experiment'], axis=1)
    _X = df.iloc[:, 0:-1].reset_index(drop=True)
    _y = df.iloc[:, -1].to_numpy()
    return _X, _y


def grid_search(model):
    models = []
    hp1 = {'MLP': [(2,), (4,), (6,), (8,), (10,),
                    (2, 2), (4, 4), (6, 6), (8, 8), (10, 10),
                    (2, 2, 2), (4, 4, 4), (6, 6, 6), (8, 8, 8), (10, 10, 10),
                    (2, 2, 2, 2), (4, 4, 4, 4), (6, 6, 6, 6), (8, 8, 8, 8), (10, 10, 10, 10),
                    (2, 2, 2, 2, 2), (4, 4, 4, 4, 4), (6, 6, 6, 6, 6), (8, 8, 8, 8, 8), (10, 10, 10, 10, 10)],
           'SVM': [ 0.001, 0.0001],
           'RF': [ 10, 50,100, 200, 500],
           'KNN':  [ 3, 4 ,5, 6, 7],
           'CDNN': [(64, 32), (128, 64)]}
    hp2 = {'MLP': ['constant'],
           'SVM': [ 10, 100, 1000],
           'RF': [ 0.9, 1.0],
           'KNN':['distance','uniform'],
           'CDNN': ['swish' , 'tanh' ,'relu' ] }
    hp3 = {
        'CDNN_dropout': [ 0.3],
        'CDNN_residual': [True]
    }
    for n in hp1[model]:
        for m in hp2[model]:
            if model == 'MLP':
                models.append(('MLP_{}_{}'.format(n, m), MLPRegressor(max_iter=10000, random_state=5,
                                                                      hidden_layer_sizes=n, learning_rate=m)))
            elif model == 'SVM':
                models.append(('SVM_{}_{}'.format(n, m), cuSVR(gamma=n, C=m)))
            elif model == 'RF':
                models.append(('RF_{}_{}'.format(n, m), cuRF(random_state=5,n_estimators=n, max_features=m)))
            elif model == 'KNN':
                models.append(('KNN_{}_{}'.format(n, m), KNeighborsRegressor(n_neighbors=n, weights=m)))
            elif model == 'CDNN':

                for dropout in hp3.get('CDNN_dropout', [0]):
                    models.append(('CDNN_{}_{}_{}'.format(n, m, dropout),
                                    KerasRegressor(build_fn=create_complex_deep_learning_model,
                                    input_dim=input_dim,
                                    hidden_layers=n,
                                    activation=m,
                                    dropout_rate=0.2,
                                    epochs=1000, batch_size=256, verbose=1)))

    return models


def create_complex_deep_learning_model(input_dim, hidden_layers, activation='relu', dropout_rate=0.3):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')  # Enable mixed precision for better GPU performance

    with tf.device('/GPU:0'):
        inputs = Input(shape=(input_dim,))
        x = Dense(hidden_layers[0], activation=activation, kernel_initializer='he_normal')(inputs)
        x = LayerNormalization()(x)
        x = Dropout(dropout_rate)(x)

        # Add residual blocks
        for layer_size in hidden_layers[1:]:
            shortcut = x  # Store previous layer for residual connection
            x = Dense(layer_size, activation=activation, kernel_initializer='he_normal')(x)
            x = LayerNormalization()(x)
            x = Dropout(dropout_rate)(x)

            # Ensure shortcut dimensions match
            if shortcut.shape[-1] != x.shape[-1]:
                shortcut = Dense(layer_size, kernel_initializer='he_normal')(shortcut)

            x = Add()([x, shortcut])  # Add residual connection

        # Output layer
        outputs = Dense(1, activation='linear', dtype='float32')(x)  # Ensure float32 output

        # Create the model
        model = Model(inputs=inputs, outputs=outputs)
        optimizer = Adam(learning_rate=1e-2, clipnorm=1.0)  # Clipping to prevent exploding gradients
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])

    return model

def compare_models(df, models):
    scoring, cv, iterations = 'neg_mean_squared_error', param['cv'], param['iterations']
    if param['scoring'] == 'r2':
        scoring = 'r2'
    # ---------------------------------
    results = pd.DataFrame()
    for i in range(iterations):
        _X_train, _y_train = split_xy(df, True)
        _X_train = _X_train.astype(np.float32)
        temp = []
        for name, model in models:
            print(name)
            cv_results = cross_val_score(model, _X_train, _y_train, cv=cv, scoring=scoring)
            cv_results = np.mean(cv_results)
            temp.append(cv_results)
        if i == 0:
            results = pd.DataFrame(temp)
        else:
            results = pd.concat([results, pd.DataFrame(temp)], axis=1, ignore_index=True)
    results['mean'] = results.mean(axis=1)
    results['std'] = results.std(axis=1)
    # ---------------------------------
    _names, _models = [], []
    for name, model in models:
        _names.append(name)
        _models.append(model)
    results['name'] = pd.Series(_names)
    results['model'] = pd.Series(_models)
    # ---------------------------------
    id_best = results['mean'].idxmax()
    _best = results.loc[id_best, 'model']
    return results, _best


def prediction(df, estimator):
    errors = pd.DataFrame()
    for i in range(param['iterations']):
        df_training, df_testing = split_data_random(df)
        _X_train, _y_train = split_xy(df_training, True)
        estimator.fit(_X_train, _y_train)
        _X_test, _y_test = split_xy(df_testing, True)
        _y_pred = estimator.predict(_X_test)
        errors.loc[i, 'r2'] = r2_score(_y_test, _y_pred)
        errors.loc[i, 'mse'] = mean_squared_error(_y_test, _y_pred)
        errors.loc[i, 'mae'] = mean_absolute_error(_y_test, _y_pred)
        errors.loc[i, 'rmse'] = np.sqrt(mean_squared_error(_y_test, _y_pred))
    _scores = [('R2', np.mean(errors['r2']), np.std(errors['r2'])),
               ('MSE', np.mean(errors['mse']), np.std(errors['mse'])),
               ('MAE', np.mean(errors['mae']), np.std(errors['mae'])),
               ('RMSE', np.mean(errors['rmse']), np.std(errors['rmse']))]
    return _scores


def split_data_exp(df, _seat_out):
    df_train = df.copy(deep=True)
    df_test = pd.DataFrame()
    for _exp in _seat_out:
        df_train = df_train.loc[df_train['Experiment'] != _exp]
        df_test = pd.concat([df_test, df.loc[df['Experiment'] == _exp]], ignore_index=True)
    return df_train, df_test


def production(df_x, df_xy):
    df_x_prod = df_x.copy(deep=True)
    df_xy_prod = df_xy.copy(deep=True)
    replicas = [i for i in df_x_prod['initial_corrosion_mm_yr'].unique()]
    df_prod_final = df_x_prod.loc[df_x_prod['initial_corrosion_mm_yr'] == replicas[0]]
    for replica in replicas:
        df_temp = df_x_prod.loc[df_x_prod['initial_corrosion_mm_yr'] == replica]
        df_prod_final = df_temp if len(df_temp) < len(df_prod_final) else df_prod_final
    _y_prod = []
    for replica in replicas:
        _y_temp = df_xy_prod.loc[df_xy_prod['initial_corrosion_mm_yr'] == replica].iloc[:, -1].to_numpy()
        _y_temp = _y_temp[:len(df_prod_final)]
        _y_prod = _y_temp.tolist() if replica == replicas[0] else [xx + yy for xx, yy in zip(_y_prod, _y_temp)]
    _y_prod = [i / len(replicas) for i in _y_prod]
    _y_prod = np.array(_y_prod)
    return df_prod_final, _y_prod


def sensitivity(df_original, df, _experiment):
    df_time = df_original.copy(deep=True)
    df_time = df_time.loc[df_time['Experiment'] == _experiment].reset_index(drop=True)
    replicas_time = df_time['initial_corrosion_mm_yr'].unique()
    df_time = df_time.loc[df_time['initial_corrosion_mm_yr'] == replicas_time[0]]
    time_hrs_sens = df_time['time_hrs_original']
    # ---------------------------------
    replicas = df['initial_corrosion_mm_yr'].unique()
    df = df.loc[df['initial_corrosion_mm_yr'] == replicas[0]]
    return df, time_hrs_sens

## Visualization Functions

In [6]:
def compare_models_box_plot(df):
    _root = 'regression/gridSearchModels'
    if not os.path.exists(_root):
        os.makedirs(_root)

    cv, iterations = param['cv'], param['iterations']

    x_axis_labels = [name for name in df['name']]
    df = df.drop(['name', 'mean', 'std', 'model'], axis=1)
    df = df.transform(lambda x: -x)
    _y_matrix = df.values.tolist()

    fig, ax = plt.subplots(1, figsize=(12, 9))

    # Use seaborn for a more professional look
    sns.boxplot(data=_y_matrix, palette="Blues", width=0.6, flierprops=dict(marker='o', markersize=5))

    # Add title and labels
    plt.title('Model Comparison using {}-fold CV ({} Replications)'.format(cv, iterations), fontsize=24, pad=20)
    plt.xticks(ticks=np.arange(len(x_axis_labels)), labels=x_axis_labels, fontsize=18, rotation=45)
    plt.yticks(fontsize=18)
    plt.ylabel('Mean Squared Error (MSE)', fontsize=20)

    # Add grid
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Add info box
    info = '{}-fold cross validation analysis \n{} replications per algorithm'.format(cv, iterations)
    plt.text(0.03, 0.96, info,
             ha='left', va='top', transform=ax.transAxes,
             fontdict={'color': 'k', 'size': 16},
             bbox={'boxstyle': 'round', 'fc': 'snow', 'ec': 'gray', 'pad': 0.5})

    plt.tight_layout()
    plt.savefig('{}/comparison.png'.format(_root))
    plt.close()

def importance_plot(df, estimator, _x, _y):
    _root = 'regression/bestModelPerformance'
    if not os.path.exists(_root):
        os.makedirs(_root)

    names = df.columns

    if hasattr(estimator, "feature_importances_"):
        # For tree-based models
        imp = estimator.feature_importances_
        indices = np.argsort(imp)

    else:
        print("Feature importance not available for this model. Using Permutation Importance instead.")

        # Compute permutation importance
        result = permutation_importance(estimator, _x, _y, scoring='r2', n_repeats=10, random_state=42)
        imp = result.importances_mean  # Mean importance scores
        indices = np.argsort(imp)

    fig, ax = plt.subplots(1, figsize=(12, 9))

    # Use a color gradient
    colors = plt.cm.Blues(np.linspace(0.3, 1, len(indices)))
    plt.barh(range(len(indices)), imp[indices], color=colors, align='center')

    # Add title and labels
    plt.title('Feature Importance', fontsize=24, pad=20)
    plt.yticks(range(len(indices)), [names[i] for i in indices], fontsize=16)
    plt.xticks(fontsize=16)
    plt.xlabel('Relative Importance', fontsize=20)

    # Add a horizontal line for visual separation
    ax.axvline(x=0, color='gray', linestyle='--', linewidth=0.8)

    plt.tight_layout()
    plt.savefig('{}/featuresImp.png'.format(_root))
    plt.close()

    excel_output(pd.DataFrame(imp), _root, file_name='rf_feature_imp', csv=False)

    permute_imp_results = permutation_importance(estimator, _x, _y, scoring='neg_mean_squared_error')
    permute_imp = permute_imp_results.importances_mean
    excel_output(pd.DataFrame(permute_imp), _root, file_name='permutation_imp', csv=False)

    return imp, permute_imp

def parity_plot(_y_test, _y_pred, _scores):
    _root = 'regression/bestModelPerformance'
    if not os.path.exists(_root):
        os.makedirs(_root)

    info = '{} = {:.3f} +/- {:.3f}\n{} = {:.3f} +/- {:.3f}\n{} = {:.3f} +/- {:.3f}\n{} = {:.3f} +/- {:.3f}'. \
        format(_scores[0][0], _scores[0][1], _scores[0][2],
               _scores[1][0], _scores[1][1], _scores[1][2],
               _scores[2][0], _scores[2][1], _scores[2][2],
               _scores[3][0], _scores[3][1], _scores[3][2])

    fig, ax = plt.subplots(1, figsize=(9, 9))

   # Convert to pandas.Series if they are numpy.ndarray
    if isinstance(_y_test, np.ndarray):
        _y_test = pd.Series(_y_test)
    if isinstance(_y_pred, np.ndarray):
        _y_pred = pd.Series(_y_pred)

    # Convert to numeric
    _y_test = pd.to_numeric(_y_test, errors='coerce')
    _y_pred = pd.to_numeric(_y_pred, errors='coerce')

    # Drop rows with NaN values
    _y_test = _y_test.dropna()
    _y_pred = _y_pred.loc[_y_test.index]  # Align _y_pred with _y_test


    print("_y_test======>",_y_test)
    print("_y_test======>",_y_pred)
    _y_test = 10 ** _y_test
    _y_pred = 10 ** _y_pred

    # Use a more professional color scheme
    plt.scatter(_y_pred, _y_test, c='blue', alpha=0.6, label='Testing set')
    a, b = min(_y_test.min(), _y_pred.min()), max(_y_test.max(), _y_pred.max())
    plt.plot([a, b], [a, b], '-', c='red', linewidth=2.0, label='y = x')

    # Add title and labels
    plt.title('Parity Plot', fontsize=24, pad=20)
    plt.xlabel('Corrosion rate (mm/year) - Predicted', fontsize=20)
    plt.ylabel('Corrosion rate (mm/year) - True', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # Add info box
    plt.text(0.03, 0.96, info,
             ha='left', va='top', transform=ax.transAxes,
             fontdict={'color': 'k', 'size': 16},
             bbox={'boxstyle': 'round', 'fc': 'snow', 'ec': 'gray', 'pad': 0.5})

    plt.legend(loc='upper right', fontsize=16, fancybox=True, shadow=True)
    plt.tight_layout()
    plt.savefig('{}/parityPlot.png'.format(_root))
    plt.close()

    df = pd.DataFrame(columns=['True_value', 'Predicted_value'])
    df['True_value'] = _y_test
    df['Predicted_value'] = _y_pred
    excel_output(df, _root, file_name='parityPlotData', csv=False)

def production_plot(df_all, df_selected, _y_pred, _mse_prod, folder_name, y_axis_scale, _exp, _seat_out):
    _root = f'regression/postProcessing/{folder_name}{y_axis_scale}'
    if not os.path.exists(_root):
        os.makedirs(_root)
    # ---------------------------------
    fig, ax = plt.subplots(1, figsize=(10, 9))
    # ---------------------------------
    df2 = df_all.copy(deep=True)
    df2 = df2.loc[df2['Experiment'] == _exp]
    replicas = df2['Description'].unique()
    n = 1
    for rep in replicas:
        df3 = df2.loc[df_all['Description'] == rep]
        _X = df3['time_hrs']
        _y = 10 ** (df3['corrosion_mm_yr'])
        _color, _zorder = 'gray', 0
        plt.scatter(_X, _y, c=_color, label=f'Replica {n}', zorder=_zorder)
        n += 1
    # ---------------------------------

    # Plot predictions (dark red color)
    df2 = df_selected.copy(deep=True)
    df2 = df2.loc[df2['Experiment'] == _exp]
    # _X_pred = df2['time_hrs_original']
    _X_pred = df2['time_hrs']
    _X_pred = _X_pred[:int(len(_X_pred) / len(replicas))]

    # Debug: Check lengths of _X_pred and _y_pred
    print(f"_exp: {str(_exp)}")
    print(f"Length of _X_pred: {len(_X_pred)}")
    print(f"Length of _y_pred: {len(_y_pred)}")

    # plt.scatter(_X_pred, 10 ** _y_pred, c='darkred', marker='^', s=[75], label='Prediction', zorder=7)
    plt.scatter(_X_pred, 10 ** _y_pred, c='#6F2DA8', marker='^', s=[75], label='Prediction', zorder=7, edgecolor='k', linewidth=0.5)
    # ---------------------------------
    if y_axis_scale == 'Log':
        plt.yscale('log')
        ax.set_ylim(0.01, 100)
    # ---------------------------------
    _info = [i for i in _seat_out]
    plt.grid(linewidth=0.5)
    x_axis_max = 25 # 10 * (1 + int(np.max(_X_pred) / 10))
    if _exp in [3,6,9, 11, 13, 17]:
        x_axis_max = 25
    elif _exp == 14:
        x_axis_max = 30
    elif _exp == 16:
        x_axis_max = 28
    x_axis_index = np.linspace(0, x_axis_max, num=6)
    ax.set_xticks(x_axis_index)
    ax.set_xlim(0, x_axis_max)
    ax.set_xticklabels(x_axis_index, fontsize=30)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    ax.set_xlabel('Time (hr)', fontsize=40, labelpad=20)
    plt.yticks(fontsize=30)
    ax.set_ylabel('Corrosion rate (mm/yr)', fontsize=40, labelpad=25)
    n_col, legend_font_size = 1, 25
    if _exp in [10, 14, 29]:
        n_col = 2
    if _exp == 14:
        legend_font_size = 18
    leg = plt.legend(loc='upper right', fontsize=legend_font_size, ncol=n_col, fancybox=True, shadow=True)
    for handle, text in zip(leg.legendHandles, leg.get_texts()):
        text.set_color(handle.get_facecolor()[0])
        if text.get_text() == 'Prediction':
            text.set_color('#6F2DA8')
    plt.tight_layout()
    plt.savefig(f'{_root}/{_info} exp{_exp} mse({_mse_prod}).png')
    plt.close()

def sensitivity_plot(df, _exp, y_axis_scale, _feature):
    _root = f'regression/sensitivityAnalysis/variation{_exp}{y_axis_scale}'
    if not os.path.exists(_root):
        os.makedirs(_root)

    # Create figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 8))

    # Define colors and markers
    colors = ['black', 'blue', 'green', 'olive', 'brown', 'purple', 'orange']  # Extended color list
    markers = ['o', 'x', '^', 's', 'D', 'P', '*']  # Extended marker list

    # Plot each feature variation
    _X = df['time_hrs']
    for i, column in enumerate(df.columns):
        if column == 'time_hrs':
            continue
        _y = 10 ** (df[column])

        # Use edgecolor only for filled markers
        if markers[i] in ['o', '^', 's', 'D', 'P', '*']:  # Filled markers
            plt.scatter(_X, _y, c=colors[i], marker=markers[i], s=100, label=column, alpha=0.8, edgecolor='k', linewidth=0.5)
        else:  # Unfilled markers (e.g., 'x')
            plt.scatter(_X, _y, c=colors[i], marker=markers[i], s=100, label=column, alpha=0.8)

    # Set y-axis scale
    if y_axis_scale == 'Log':
        plt.yscale('log')
        ax.set_ylim(0.01, 100)

    # Add grid and labels
    plt.grid(True, linestyle='--', alpha=0.6)
    x_axis_max = 10 * (1 + int(np.max(_X) / 10))
    if _exp == 6:
        x_axis_max = 40
    elif _exp in [11, 13, 17, 18, 19]:
        x_axis_max = 25
    elif _exp == 14:
        x_axis_max = 30
    elif _exp == 16:
        x_axis_max = 15

    x_axis_index = np.linspace(0, x_axis_max, num=6)
    ax.set_xticks(x_axis_index)
    ax.set_xlim(0, x_axis_max)
    ax.set_xticklabels(x_axis_index, fontsize=18)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    ax.set_xlabel('Time (hr)', fontsize=22, labelpad=10)
    ax.set_ylabel('Corrosion rate (mm/yr)', fontsize=22, labelpad=10)
    plt.yticks(fontsize=18)

    # Add legend
    plt.legend(loc='upper right', fontsize=16, ncol=1, fancybox=True, shadow=True, framealpha=0.9)

    # Add title
    plt.title(f'Sensitivity Analysis: {_feature} (Experiment {_exp})', fontsize=24, pad=20)

    # Save and close
    plt.tight_layout()
    plt.savefig(f'{_root}/{_feature}.png', dpi=300)
    plt.close()

    # Save data to Excel
    excel_output(df, _root, file_name=_feature, csv=False)



## Main (REGRESSION PROBLEM)

In [ ]:
# reading data
dataAll, n_exp = read_data('SequentialDataInhibitor_large', new=False)

# data summary (one-time output)
if param['summary']:
    summary_data(df=dataAll)


# # pre-processing data
dataSelected = select_features(dataAll)
inhibitor = encode_data(dataSelected)


# grid-search to find the best hyper-parameters of each algorithm (one-time output)
if param['grid_search']:
    root = 'regression/gridSearchModels'
    if not os.path.exists(root):
        os.makedirs(root)
    # ---------------------------------
    best_models = {}
    df_scores = pd.DataFrame()
    for algorithm in ['MLP','CDNN','SVM', 'RF','KNN']:  #
        print(algorithm)
        algorithms = grid_search(algorithm)
        scores, best = compare_models(inhibitor, algorithms)
        best_models[algorithm] = best
        df_scores['{}_mean'.format(algorithm)], df_scores['{}_std'.format(algorithm)] = scores['mean'], scores['std']
        printOut = pd.DataFrame(algorithms)
        printOut['mean'], printOut['std'] = [-x for x in scores['mean']], scores['std']
        excel_output(printOut, root, file_name='{}'.format(algorithm), csv=False)
    models_reg = [
                  ('MLP', best_models['MLP']),
                  ('CDNN', best_models['CDNN']),
                  ('SVM', best_models['SVM']),
                  ('RF', best_models['RF']),
                  ('KNN', best_models['KNN'])]
else:
    # The following models with the set hyper-parameters are found after the one time grid searching
    models_reg = [
                  ('MLP', MLPRegressor(hidden_layer_sizes=(8, 8, 8, 8), max_iter=10000)),
                  ('CDNN', KerasRegressor(build_fn=create_complex_deep_learning_model, input_dim=17,
                                     hidden_layers=(512, 256, 128), activation='relu',
                                     dropout_rate=0.5,
                                     epochs=100, batch_size=64, verbose=1)),
                  ('SVM', cuSVR(C=1000, gamma=1)),
                  ('RF', cuRF(max_features=0.5, n_estimators=300)),
                  ('KNN', KNeighborsRegressor(n_neighbors=3, weights='distance'))]

# comparing four ML models
_best_reg = models_reg[2][1]   ##   _best_reg = models_reg[0][1]
if param['compare']:
    scores_reg, _best_reg = compare_models(inhibitor, models_reg)
    compare_models_box_plot(scores_reg)
    excel_output(scores_reg, 'regression/gridSearchModels', file_name='comparison', csv=False)
best_reg = _best_reg

# features importance
if param['importance']:
    X, y = split_xy(inhibitor, True)
    best_reg.fit(X, y)
    feature_importance, permute_importance = importance_plot(inhibitor, best_reg, X, y)

# parity plot for 25% of the data
if param['parity']:
    training_reg, testing_reg = split_data_random(inhibitor)
    X_train, y_train = split_xy(training_reg, True)
    best_reg.fit(X_train, y_train)
    X_test, y_test = split_xy(testing_reg, True)
    y_pred = best_reg.predict(X_test)
    scores_pred = prediction(inhibitor, best_reg)
    parity_plot(y_test, y_pred, scores_pred)
    excel_output(X_train, 'regression/bestModelPerformance', file_name='trainFeatureMatrixNorm', csv=False)

# testing the model when 4 experiment randomly selected to seat out
if param['production']:
    experiments = inhibitor['Experiment'].unique()
    seatOuts = [[int(i) for i in np.random.choice(a=experiments, size=20, replace=False)]]
    for seatOut in seatOuts:
        print(seatOut)
        training_test, testing_test = split_data_exp(inhibitor, seatOut)
        X_train, y_train = split_xy(training_test, True)
        best_reg.fit(X_train, y_train)
        for exp in seatOut:
            testing_temp = testing_test.loc[testing_test['Experiment'] == exp].reset_index(drop=True)
            X_test, y_test = split_xy(testing_temp, False)
            X_prod, y_prod = production(X_test, testing_temp)
            X_prod = X_prod[:int(len(testing_temp))]
            mse_prod = 0
            for iteration in range(param['iterations']):
                y_pred = best_reg.predict(X_prod)
                mse_prod += mean_squared_error(y_prod, y_pred)
            y_pred = best_reg.predict(X_prod)
            mse_prod = mse_prod / param['iterations']
            production_plot(dataAll, dataSelected, y_prod, mse_prod, f'testingTheModel/{seatOut}', 'Log', exp, seatOut)
            production_plot(dataAll, dataSelected, y_prod, mse_prod, f'testingTheModel/{seatOut}', 'Norm', exp, seatOut)


# sensitivity analysis
if param['sensitivity']:
    experiments = [int(i) for i in inhibitor['Experiment'].unique()]
    # experiments = [11]
    experiment = [i for i in np.random.choice(a=experiments, size=1, replace=False)]
    features_reg = {'CI': [['EC1612A', 'CORR12148SP'], [0.0, 0.0], 'Corrosion inhibitor', 'Inhibitor type', ''],
                    'pH': [['Controlled=6', 'Uncontrolled'], [0.0, 0.0], 'pH', 'pH', ''],
                    'Brine_Type': [['TH', 'Galapagos'], [0.0, 0.0], 'Brine type', 'Brine type', ''],
                    'Pressure_bar_CO2': [[0.5, 5, 12, 50], [4.51, 3.15],
                                         'CO2 partial pressure', '$p_{CO2}$', 'bar'],
                    'Temperature_C': [[30, 90, 110, 132], [106.69, 19.34],
                                      'Temperature', 'T', '$^oC$'],
                    'Shear_Pa': [[20, 50, 100, 200, 300], [32.85, 56.01],
                                 'Wall shear stress', '$\u03C4$', 'Pa'],
                    'Brine_Ionic_Strength': [[0.5, 1.5, 2.5, 5, 10], [0.87, 0.62],
                                             'Brine ionic strength', '$C_{Brine}$', 'M'],
                    'concentration_ppm': [[10, 100, 200, 300, 1000], [190.21, 131.99],
                                          'CI concentration', '$C_{CI}$', 'ppm']}
    for experiment in experiments:
        print(experiment)
        training_sens, testing_sens = split_data_exp(inhibitor, [experiment])
        X_train, y_train = split_xy(inhibitor, True)

        y_train = y_train.astype(float)


        best_reg.fit(X_train, y_train)
        testing_sens, time_sens = sensitivity(dataSelected, testing_sens, experiment)
        for key in features_reg:
            print(key)
            first = True
            sensitivity_df = pd.DataFrame(index=range(len(testing_sens)))
            sensitivity_df['time_hrs'] = time_sens
            for value in features_reg[key][0]:
                testing_temp = testing_sens.copy(deep=True)
                if first and key in ['CI', 'pH', 'Brine_Type']:
                    testing_temp['{}_{}'.format(key, features_reg[key][0][0])] = [1.0] * len(testing_sens)
                    testing_temp['{}_{}'.format(key, features_reg[key][0][1])] = [0.0] * len(testing_sens)
                    first = False
                elif key in ['CI', 'pH', 'Brine_Type']:
                    testing_temp['{}_{}'.format(key, features_reg[key][0][0])] = [0.0] * len(testing_sens)
                    testing_temp['{}_{}'.format(key, features_reg[key][0][1])] = [1.0] * len(testing_sens)
                else:
                    key_mean, key_std = np.mean(dataSelected[key]), np.std(dataSelected[key])
                    zero_norm = (0 - key_mean) / float(key_std)
                    value_norm = (value - key_mean) / float(key_std)
                    if key != 'concentration_ppm':
                        testing_temp[key] = [value_norm] * len(testing_sens)
                    else:
                        for v in range(len(testing_temp)):
                            if testing_temp.loc[v, 'concentration_ppm'] != 0:
                                testing_temp.loc[v, 'concentration_ppm'] = value
                X_sens = testing_temp.drop(['Description', 'Experiment', 'corrosion_mm_yr'], axis=1)
                y_sens = best_reg.predict(X_sens)
                value = 130 if value == 132 else value
                if value == 'EC1612A':
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], 'CI-1', features_reg[key][4])] = y_sens
                elif value == 'CORR12148SP':
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], 'CI-2', features_reg[key][4])] = y_sens
                elif value == 'Controlled=6':
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], 'controlled 6',
                                                       features_reg[key][4])] = y_sens
                elif value == 'Uncontrolled':
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], 'uncontrolled',
                                                       features_reg[key][4])] = y_sens
                elif value == 'TH':
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], 'A', features_reg[key][4])] = y_sens
                elif value == 'Galapagos':
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], 'B', features_reg[key][4])] = y_sens
                else:
                    sensitivity_df['{} = {} {}'.format(features_reg[key][3], value, features_reg[key][4])] = y_sens
            sensitivity_plot(sensitivity_df, experiment, 'Log', features_reg[key][2])
            sensitivity_plot(sensitivity_df, experiment, 'Norm', features_reg[key][2])


print('DONE!')